In [ ]:
import glob
import pathlib
import pickle
import requests
import tarfile
import time

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from sklearn import decomposition, ensemble, linear_model, metrics, tree, utils
from sklearn import model_selection, pipeline, preprocessing

# Working with Real Data

When you are just getting started with machine learning it is best to experiment with real-world data (as opposed to artificial data). The following are some good resources of open-source data that you can use for practice or research.

* [University of California-Irvine Machine Learning Repository](http://archive.ics.uci.edu/ml/)
* [Kaggle](https://www.kaggle.com/datasets),
* [OpenDataMonitor](http://opendatamonitor.eu/),
* [Wikipedia's list of Machine Learning datasets](https://en.wikipedia.org/wiki/List_of_datasets_for_machine-learning_research)
* [Datasets subreddit](https://www.reddit.com/r/datasets/),
* [Quora's list of open datasets](https://www.quora.com/Where-can-I-find-large-datasets-open-to-the-public)

Major cloud providers all have repositories of publically available datasets.,

* [Open Data on AWS](https://registry.opendata.aws/),
* [Open Data on GCP](https://cloud.google.com/public-datasets/),
* [Open Data on Azure](https://azure.microsoft.com/en-us/services/open-datasets/),
    
Finally, [Pandas DataReader](https://pydata.github.io/pandas-datareader/) provides a unified API to a [number of datasets](https://pydata.github.io/pandas-datareader/remote_data.html). Note that many of these data sources require you to create an account and get an API key.

## CIFAR-10 Dataset

The original [CIFAR-10](http://www.cs.toronto.edu/~kriz/cifar.html) dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5000 images from each class.

In [ ]:
CLASS_LABELS = {
    0: "airplane",
    1: "automobile",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck"
}

### Download and extract the data

In [ ]:
DATA_DIR = pathlib.Path("../data/")
RAW_DATA_DIR = DATA_DIR / "cifar-10"
URL = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"


RAW_DATA_DIR.mkdir(parents=True, exist_ok=True)

with open(RAW_DATA_DIR / "cifar-10-python.tar.gz", "wb") as f:
    response = requests.get(URL)
    f.write(response.content)

with tarfile.open(RAW_DATA_DIR / "cifar-10-python.tar.gz", "r:gz") as f:
    f.extractall(RAW_DATA_DIR)


### Load the data

We will load the data using the [Pandas](https://pandas.pydata.org/) library. Highly recommend the most recent edition of [*Python for Data Analysis*](https://learning.oreilly.com/library/view/python-for-data/9781491957653/) by Pandas creator Wes Mckinney for anyone interested in learning how to use Pandas.

In [ ]:
data = []
labels = []
filepaths = glob.glob("../data/cifar-10/cifar-10-batches-py/*_batch*")
for filepath in sorted(filepaths):
    with open(filepath, "rb") as f:
        batch = pickle.load(f, encoding="latin1")
        data.append(batch["data"])
        labels.extend(batch["labels"])

# each image has 3 channels with height and width of 32 pixels
features = pd.DataFrame(
    np.vstack(data),
    columns=[f"p{i}" for i in range(3 * 32 * 32)],
    dtype="uint8",
)
target = pd.Series(labels, dtype="uint8", name="labels")

### Explore the data

In [ ]:
features.info()

In [ ]:
features.head()

In [ ]:
features.tail()

In [ ]:
features.describe()

In [ ]:
_ = (target.value_counts()
           .sort_index()
           .plot(kind="bar"))

### Visualize the data

In [ ]:
fig, axes = plt.subplots(10, 10, sharex=True, sharey=True, figsize=(15, 15))
for i in range(10):
    for j in range(10):
        m, _ = features.shape
        k = np.random.randint(m)
        img = (features.loc[k, :]
                       .to_numpy()
                       .reshape((3, 32, 32))
                       .transpose(1, 2, 0))
        _ = axes[i, j].imshow(img)
        _ = axes[i, j].set_title(CLASS_LABELS[target[k]])

fig.suptitle("Random CIFAR-10 images", x=0.5, y=1.0, fontsize=25)
fig.tight_layout()

### Write the data to disk (optional)

If you want you can write out the features and the target to disk.

In [ ]:
_ = (features.join(target)
             .to_csv(RAW_DATA_DIR / "cifar-10.csv", index=False))

# Look at the Big Picture

Our goal over these two hands-on workshops will be to build a machine learning modeling pipeline that is capable of classifying images. Today we will mostly focus on classical machine learning algorithms implemented in Scikit-Learn; tomorrow we will revist the same problem using deep learning algorithms implemented in PyTorch. By the time you have finished this two-day workshop you should understand how to build a machine learning application capable of classifying images and be ready to apply what you have learned to a new dataset.

This morning we will mostly focus on getting the data and exploring the data to gain new insights. Believe it or not these initial steps are what data scientists and machine learning engineers spend the majority of their time doing! This afternoon we will prepare our data for machine learning, see how to fit a variety of machine learning models to our dataset and shortlist a few candidate models for further analysis. We will then use hyper-parameter tuning to improve the performance of our shortlisted models to arrive at an overall best model. We will finish with a discussion of how to present the results of your model and talk about some of the aspects of deploying a trained model to make predictions.

## Framing the problem

### What is the business/research objective?

Typically building the model is not the overall objective but rather the model itself is one part of a larger process used to answer a business/research question. Knowing the overall objective is important because it will determine your choice of machine learning algorithms to train, your measure(s) of model performance, and how much time you will spend tweaking the hyper-parameters of your model.

In our example today, the overall business/research objective is to make money from serving targeted ads to users of a social media application based on the images that they post. Our image classication model might just be one of potentially many other models whose predictions are taken as inputs into another machine learning model that will be used to determine which ads to show to users. 

### What is the current solution?

Always a good idea to know what the current solution to the problem you are trying to solve. Current solution gives a benchmark for performance. Note that the current "best" solution could be very simple or could be very sophisticated. Understanding the current solution helps you think of a good place to start. Example: suppose that the current solution for predicting the price of a house in a given census block is to ignore all the demographic information and predict a simple average of house prices in nearby census blocks. In this case it would probably not make sense to start building a complicated deep learning model to predict housing prices. However, if the current solution was a tuned gradient boosted machine then it probably would not make sense to try a much simpler linear regression model.

With all this information, you are now ready to start designing your system. First, you need to frame the problem by answering the following questions.

* Is our problem supervised, unsupervised, or reinforcement learning?
* Is our problem a classification task, a regression task, or something else? If our problem is a classification task are we trying to classify samples into 2 categories (binary classification) or more than 2 (multi-class classification) categories? If our problem is a regression task, are we trying to predict a single value (univariate regression) or multiple values (multivariate regression) for each sample?
* Should you use batch learning or online learning techniques?


### Exercise: Selecting a metric

Scikit-Learn has a number of different [possible metrics](https://scikit-learn.org/stable/modules/model_evaluation.html) that you can choose from (or you can create your own custom metric if required). Can you find a few metrics that seems appropriate for our image classification model?

# Creating a Test Dataset

Before we look at the data any further, we need to create a test set, put it aside, and never look at it (until we are ready to test our trainined machine learning model!). Why? We don't want our machine learning model to memorize our dataset (this is called overfitting). Instead we want a model that will generalize well (i.e., make good predictions) for inputs that it didn't see during training. To do this we hold split our dataset into training and testing datasets. The training dataset will be used to train our machine learning model(s) and the testing dataset will be used to make a final evaluation of our machine learning model(s).

## If you might refresh data in the future...

...then you want to use some particular hashing function to compute the hash of a unique identifier for each observation of data and include the observation in the test set if resulting hash value is less than some fixed percentage of the maximum possible hash value for your algorithm. This way even if you fetch more data, your test set will never include data that was previously included in the training data.

In [ ]:
import zlib


def in_testing_data(identifier, test_size):
    _hash = zlib.crc32(bytes(identifier))
    return _hash & 0xffffffff < test_size * 2**32


def split_train_test_by_id(data, test_size, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda identifier: in_testing_data(identifier, test_size))
    return data.loc[~in_test_set], data.loc[in_test_set]


## If this is all the data you will ever have...

...then you can just set a seed for the random number generator and then randomly split the data. Scikit-Learn has a [`model_selection`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) module that contains tools for splitting datasets into training and testing sets.

In [ ]:
model_selection.train_test_split?

In [ ]:
SEED = 42
SEED_GENERATOR = np.random.RandomState(SEED)


def generate_seed():
    return SEED_GENERATOR.randint(np.iinfo("uint16").max)

In [ ]:
# split the dataset into training and testing data
_seed = generate_seed()
_random_state = np.random.RandomState(_seed)
train_features, test_features, train_target, test_target = model_selection.train_test_split(
    features,
    target,
    test_size=1e-1,
    random_state=_random_state
)

In [ ]:
train_features.info()

In [ ]:
train_features.head()

In [ ]:
train_target.head()

In [ ]:
train_features.describe()

Again, if you want to you can write out the train and test sets to disk to avoid having to recreate them later.

In [ ]:
_ = (train_features.join(train_target)
                   .to_csv(RAW_DATA_DIR / "train.csv", index=False))

_ = (test_features.join(test_target)
                   .to_csv(RAW_DATA_DIR / "test.csv", index=False))

# Prepare the data for machine learning algorithms

Best practice is to write functions to automate the process of preparing your data for machine learning. Why?

* Allows you to reproduce these transformations easily on any dataset.
* You will gradually build a library of transformation functions that you can reuse in future projects.
* You can use these functions in a live system to transform the new data before feeding it to your algorithms.
* This will make it possible for you to easily experiment with various transformations and see which combination of transformations works best.

We are working with an benchmark dataset that has already been prepared for analysis (mostly!). You should be aware that academic benchmark datasets are not very representative of the type of datasets that you will encounter in most practical applications. 

## Feature Scaling

Machine learning algorithms typically don’t perform well when the input numerical attributes have very different scales. The simplest approach is to rescale features so that they all reside within the same range (typically between 0 and 1). This approach is implemented in Scikit-Learn by the [`preprocessing.MinMaxScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler) class.

In [ ]:
preprocessing.MinMaxScaler?

In [ ]:
# hyper-parameters
_preprocessing_hyperparameters = {
    "feature_range": (0, 1),
    "copy": True,
    "clip": False,
}
preprocessor = preprocessing.MinMaxScaler(**_preprocessing_hyperparameters)

In [ ]:
preprocessed_train_features = preprocessor.fit_transform(train_features)

In [ ]:
preprocessed_train_features.shape

In [ ]:
preprocessed_train_features[:, :5]

In [ ]:
preprocessed_train_features.min(axis=0)

In [ ]:
preprocessed_train_features.mean(axis=0)

In [ ]:
preprocessed_train_features.max(axis=0)

The `preprocessing.MinMaxScaler` and the `preprocessing.StandardScaler` classes are the first Scikit-Learn `Transformer` classes that we have encountered. As such now is a good to to discuss the Scikit-Learn application programming interface (API). The [Scikit-Learn API](https://scikit-learn.org/stable/modules/classes.html) is one of the best designed API's around and has heavily influenced API design choices of other libraries in the Python Data Science and Machine Learning ecosystem, in particular [Dask](https://dask.org/) and [NVIDIA RAPIDS](https://rapids.ai/). Familiarly with the Scikit-Learn API will make it easier for you to get started with these libraries.

The Scikit-Learn API is built around the following key concepts.

* Estimators: Any object that can estimate some parameters based on a dataset is called an estimator (e.g., an `preprocessing.MinMaxScaler` is an estimator). The estimation itself is performed by the `fit` method, and it takes only a dataset as a parameter (or two for supervised learning algorithms; the second dataset contains the labels). Any other parameter needed to guide the estimation process is considered a hyperparameter (such as the `feature_range` parameter in `preprocessing.MinMaxScaler`), and it must be set as an instance variable (generally via a constructor parameter).

* Transformers: Some estimators (such as an `preprocessing.MinMaxScaler`) can also transform a dataset; these are called transformers. Once again, the API is simple: the transformation is performed by the transform method with the dataset to transform as a parameter. It returns the transformed dataset. This transformation generally relies on the learned parameters. All transformers also have a convenience method called `fit_transform` that is equivalent to calling `fit` and then `transform` (but sometimes `fit_transform` is optimized and runs much faster).

* Predictors: Finally, some estimators, given a dataset, are capable of making predictions; they are called predictors. A predictor has a `predict` method that takes a dataset of new instances and returns a dataset of corresponding predictions. It also has a score method that measures the quality of the predictions, given a test set (and the corresponding labels, in the case of supervised learning algorithms).

All of an estimator’s hyperparameters are accessible directly via public instance variables (e.g., `preprocessor.feature_range`), and all the estimator’s learned parameters are accessible via public instance variables with an underscore suffix (e.g., `preprocessor.scale_`). Finally, Scikit-Learn provides reasonable default values for most parameters which makes it easy to quickly create a baseline working system.

### Exercise: MinMaxScaler vs StandardScaler

An alternative approach is to rescale features so that they all have zero mean and unit standard deviation. This approach, which is also called standardization, is particularly useful when attributes/features have outliers and when downstream machine learning algorithms assume that attributes/features have a Gaussian or Normal distribution. 

Create an instance of the [`preprocessing.StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) class and use it to rescale the training dataset. Compare the two different rescaled versions of the dataset. Which of the two methods do you prefer?

In [ ]:
# insert your code here!

As with all the transformations, it is important to fit the scalers to the training data only, not to the full dataset (including the test set). Only then can you use them to transform the training set and the test set (and new data).

## Feature Engineering

Feature engineering is one of the most important parts of any machine learning project. There are two main tasks in feature engineering.

* Feature selection: selecting the best subset of features for training. 
* Feature extraction: combining existing features to produce new features for training.
* Feature creation: finding additional data sources to use as features.

Feature engineering is often the most labor intensive part of building a machine learning pipeline and often requires extensive expertise/domain knowledge relevant to the problem at hand. Recently packages such as [featuretools](https://www.featuretools.com/) have been developed to (partially) automate the process of feature engineering.

The success of deep learning in various domains is in significant part due to the fact that deep learning models are able to automatically engineer features that are most useful for solving certain machine learning tasks. In effect deep learning replaces the expensive to acquire expertise/domain knowledge required to hand-engineer predictive features. 

A recent example that demonstrates that power of automated feature engineering is [Space2vec](https://medium.com/dessa-news/space-2-vec-fd900f5566), a deep learning based supernovae classifier developed by machine learning engineers with no expertise in Astronomy that was able to outperform the machine learning solution developed by NERSC scientists. The machine learning pipeline developed by NERSC scientists, called [AUTOSCAN](https://portal.nersc.gov/project/dessn/autoscan/), was a significant improvement over the previous solution which relied on manual classification of supernovae by astronomers. However, in order to achieve such high accuracy, the NERSC solution relied on a dataset of hand-engineered features developed by astronomers with over a century of combined training and expertise in the domain. The deep learning algorithm used by space2vec could be applied directly to the raw image data and did not rely on any hand-engineered features.

### Feature extraction using Principal Component Analysis (PCA)

In [ ]:
decomposition.PCA?

In [ ]:
_seed = generate_seed()

# hyper-parameters
_pca_hyperparameters = {
    "n_components": 250,
    "random_state": np.random.RandomState(_seed),
    "svd_solver": "randomized",
    "whiten": False,
}

decomposer = decomposition.PCA(**_pca_hyperparameters)
engineered_train_features = decomposer.fit_transform(preprocessed_train_features)

In [ ]:
engineered_train_features.shape

In [ ]:
engineered_train_features[:, :5]

In [ ]:
engineered_train_features.mean(axis=0)

In [ ]:
engineered_train_features.std(axis=0)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
_ = ax.scatter(
    engineered_train_features[:, 0],
    engineered_train_features[:, 1],
    c=train_target,
    alpha=0.05
)
_ = ax.set_xlabel("Component 1", fontsize=15)
_ = ax.set_ylabel("Component 2", fontsize=15)
_ = ax.set_title(type(decomposer))
_ = ax.grid(True)

### Exercise: To whiten, or not to whiten?

Take a close look at the doc string for the `decomposition.PCA` algorithm. What happens if you set `n_components` to a number between 0 and 1 (i.e., `n_components=0.95`)? Why might you want to do this? What does setting `whiten=True` do to the output of the algorithm? Re-run the PCA algorithm above setting `whiten=True` to confirm your answer. Why might you want to set `whiten=True`? 

In [ ]:
# insert code here!

## Transformation pipelines

As you can see creating preprocessing pipelines involves quite a lot of steps and each of the steps needs to be executed in the correct order. Fortunately Scikit-Learn allows you to combine estimators together to create [pipelines](https://scikit-learn.org/stable/modules/compose.html#combining-estimators). We can encapsulate all of the preprocessing logic for our numeric attributes as well as the preprocessing logic for our non-numeric attributes into separate instances of the [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) class.

The `Pipeline` constructor takes a list of name/estimator pairs defining a sequence of steps. All but the last estimator must be transformers (i.e., they must have a `fit_transform` method). The names can be anything you like (as long as they are unique). Later we will see how to access the parameters of pipelines using these names when we discuss hyperparameter tuning.

In [ ]:
_seed = generate_seed()

# hyper-parameters
_min_max_scaler_hyperparameters = {
    "feature_range": (0, 1)
}

_pca_hyperparameters = {
    "n_components": 250,
    "random_state": np.random.RandomState(_seed),
    "svd_solver": "randomized",
    "whiten": False,
}

# default Pipeline constructor
preparation_pipeline = pipeline.Pipeline(
    [
        ("minmaxscaler", preprocessing.MinMaxScaler(**_min_max_scaler_hyperparameters)),
        ("pca", decomposition.PCA(**_pca_hyperparameters)),
    ],
    verbose=True,
)

In [ ]:
_seed = generate_seed()

# hyper-parameters
_min_max_scaler_hyperparameters = {
    "feature_range": (0, 1)
}

_pca_hyperparameters = {
    "n_components": 250,
    "random_state": np.random.RandomState(_seed),
    "svd_solver": "randomized",
    "whiten": False,
}

# alternative constructor that is equivalent to the above!
preparation_pipeline = pipeline.make_pipeline(
    preprocessing.MinMaxScaler(**_min_max_scaler_hyperparameters),
    decomposition.PCA(**_pca_hyperparameters),
    verbose=True,
)

In [ ]:
prepared_train_features = preparation_pipeline.fit_transform(train_features)

In [ ]:
prepared_train_features.min(axis=0)

In [ ]:
prepared_train_features.mean(axis=0)

In [ ]:
prepared_train_features.max(axis=0)

### Exercise: Feature scaling prior to using PCA?

Do you think that it is necessary to rescale the raw features prior to using PCA? Why or why not?

# Select and train a model

At last! You framed the problem, you got the data and explored it, you sampled a training set and a test set, and you wrote transformation pipelines to clean up and prepare your data for machine learning algorithms automatically. You are now ready to select and train a Machine Learning model. You might have been wondering if we were every going to make it to this point! Fact is, most of your time developing machine learning solutions to real-world problems will not be spent training machine learning models: most of your time will be spent preparing the data for machine learning algorithms and most of the computer time will be spent training the machine learning models.

## Training and evaluating on the training dataset

### Linear models

In [ ]:
# hyper-parameters
_min_max_scaler_hyperparameters = {
    "feature_range": (0, 1)
}

preparation_pipeline = pipeline.make_pipeline(
    preprocessing.MinMaxScaler(**_min_max_scaler_hyperparameters),
    verbose=True,
)

In [ ]:
prepared_train_features = preparation_pipeline.fit_transform(train_features)

In [ ]:
_seed = generate_seed()
_classifier_hyperparameters = {
    "fit_intercept": True,
    "loss": "log",
    "penalty": None,
    "random_state": np.random.RandomState(_seed),
}
estimator = linear_model.SGDClassifier(**_classifier_hyperparameters)
_ = estimator.fit(prepared_train_features, train_target)

In [ ]:
predictions = estimator.predict(prepared_train_features)

In [ ]:
predictions

Congrats! You have fit your first machine learning model using Scikit-Learn and made some predictions. Now let's see how good those predictions really are.

### Evaluation metrics

#### Confusion matrix

In [ ]:
confusion_matrix = metrics.confusion_matrix(
    train_target,
    predictions,
)
confusion_matrix

In [ ]:
# visualize the normalized confusion matrix
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
_normalized_confusion_matrix = confusion_matrix / confusion_matrix.sum(axis=1, keepdims=True)
np.fill_diagonal(_normalized_confusion_matrix, 0)
_ = ax.matshow(_normalized_confusion_matrix)
_ = ax.set_xlabel("Predicted Class", fontsize=15)
_ = ax.set_ylabel("Actual Class", fontsize=15)
plt.show()

#### Precision

In [ ]:
metrics.precision_score(
    train_target,
    predictions,
    average="macro",
)

#### Recall

In [ ]:
metrics.recall_score(
    train_target,
    predictions,
    average="macro",
)

#### $F_1$ Score

In [ ]:
metrics.f1_score(
    train_target,
    predictions,
    average="macro",
)

#### Receiver Operating Characteristic (ROC) Area Under the Curve (AUC) Score

In [ ]:
_scores = estimator.predict_proba(prepared_train_features)
metrics.roc_auc_score(
    train_target,
    _scores,
    average="macro",
    multi_class="ovo",
)

#### Classification report

In [ ]:
_report = metrics.classification_report(
    train_target,
    predictions,
)
print(_report)

### Accelerating training

How can we speed up training? Use more cores and/or some other [dimensionality reduction](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.decomposition) technique to reduce the dimension of the feature space.

In [ ]:
_seed = generate_seed()
_pca_hyperparameters = {
    "n_components": 250,
    "random_state": np.random.RandomState(_seed),
    "svd_solver": "randomized",
    "whiten": True
}

# use PCA to reduce dimensionality and standardize features
_preparation_pipeline = pipeline.make_pipeline(
    decomposition.PCA(**_pca_hyperparameters),
    verbose=True,
)
prepared_train_features = _preparation_pipeline.fit_transform(train_features)

In [ ]:
train_features.shape

In [ ]:
prepared_train_features.shape

In [ ]:
prepared_train_features.mean(axis=0)

In [ ]:
prepared_train_features.std(axis=0)

In [ ]:
_seed = generate_seed()
_classifier_hyperparameters = {
    "fit_intercept": False,
    "loss": "log",
    "n_jobs": -1,
    "penalty": None,
    "random_state": np.random.RandomState(_seed),
}
_estimator = linear_model.SGDClassifier(**_classifier_hyperparameters)
_ = _estimator.fit(prepared_train_features, train_target)

# make predictions
_predictions = _estimator.predict(prepared_train_features)

# report the accuracy on the training data
_report = metrics.classification_report(
    train_target,
    _predictions,
)
print(_report)

### Exercise: experiment with different loss functions

In [ ]:
linear_model.SGDClassifier?

In [ ]:
_seed = generate_seed()
_classifier_hyperparameters = {
    "fit_intercept": False,
    "loss": "hinge", # change this!
    "n_jobs": -1,
    "penalty": None,
    "random_state": np.random.RandomState(_seed),
}
_estimator = linear_model.SGDClassifier(**_classifier_hyperparameters)
_ = _estimator.fit(prepared_train_features, train_target)

# make predictions
_predictions = _estimator.predict(prepared_train_features)

# report the accuracy on the training data
_report = metrics.classification_report(
    train_target,
    _predictions,
)
print(_report)

### Exercise: experiment with different penalties

In [ ]:
linear_model.SGDClassifier?

In [ ]:
_seed = generate_seed()
_classifier_hyperparameters = {
    "alpha": 1e-4, # try changing this!
    "fit_intercept": False,
    "l1_ratio": 0.15, # only used for penalty=elastic_net
    "loss": "log",
    "n_jobs": -1,
    "penalty": None, # try changing this!
    "random_state": np.random.RandomState(_seed),
}
_estimator = linear_model.SGDClassifier(**_classifier_hyperparameters)
_ = _estimator.fit(prepared_train_features, train_target)

# make predictions
_predictions = _estimator.predict(prepared_train_features)

# report the accuracy on the training data
_report = metrics.classification_report(
    train_target,
    _predictions,
)
print(_report)

### Mini-batch gradient descent

Since we talked about the difference between stochastic, batch, and mini-batch gradient descent in the lectures I wanted you to see how to implement mini-batch gradient descent in Scikit-Learn. You will see much more of this idea in the deep learning hands on session so we will not spend too much time on it now.

In [ ]:
_seed = generate_seed()
_random_state = np.random.RandomState(_seed)

n_epochs = 2
batch_size = 128
X = prepared_train_features
y = train_target
m, _ = X.shape

# define your estimator
_classifier_hyperparameters = {
    "alpha": 1e-4,
    "fit_intercept": False,
    "l1_ratio": 0.15,
    "learning_rate": "optimal",
    "loss": "log",
    "n_jobs": -1,
    "penalty": None,
    "random_state": _random_state,
    "warm_start": True,
}
estimator = linear_model.SGDClassifier(**_classifier_hyperparameters)

# nested for loops implement the training
for _ in range(n_epochs):

    # shuffle the dataset before every training epoch
    shuffled_indices = _random_state.permutation(m)
    _X, _y = X[shuffled_indices], y.iloc[shuffled_indices]

    for batch_ixs in utils.gen_batches(m, batch_size):
        _ = estimator.partial_fit(_X[batch_ixs], _y[batch_ixs], classes=y.unique())


In [ ]:
# make predictions
_predictions = estimator.predict(prepared_train_features)

# report the accuracy on the training data
_report = metrics.classification_report(
    train_target,
    _predictions,
)
print(_report)

### Discussion

What is going on here? Are we underfitting? Are we overfitting? If you think that we are underfitting, then what could we do to try and get the model to overfit? If we are overfitting, what could we do to get the model to underfit?

In [ ]:
CV_FOLDS = 3

In [ ]:
model_selection.learning_curve?

In [ ]:
_seed = generate_seed()
_random_state = np.random.RandomState(_seed)
_classifier_hyperparameters = {
    "fit_intercept": False,
    "loss": "log",
    "n_jobs": -1,
    "penalty": None,
    "random_state": _random_state,
}
_estimator = linear_model.SGDClassifier(**_classifier_hyperparameters)
_ = _estimator.fit(prepared_train_features, train_target)

train_sizes, train_scores, val_scores = model_selection.learning_curve(
    _estimator,
    prepared_train_features,
    train_target,
    cv=CV_FOLDS,
    n_jobs=-1,
    random_state=_random_state,
    scoring="accuracy",
    train_sizes=np.linspace(0.1, 1.0, 15),
    verbose=1,
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
ax.plot(train_sizes, train_scores.mean(axis=1), "r-+", linewidth=2, label="train")
ax.plot(train_sizes, val_scores.mean(axis=1), "b-", linewidth=3, label="valid")
ax.set_xlabel("Training set size", fontsize=15)
ax.set_ylabel("Accuracy", fontsize=15)
ax.grid()
ax.legend()

plt.show()

### Decision Trees

[Decision Trees](https://scikit-learn.org/stable/modules/tree.html) are a non-parametric supervised learning method used for [classification](https://scikit-learn.org/stable/modules/tree.html#tree-classification) and [regression](https://scikit-learn.org/stable/modules/tree.html#tree-regression). The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features. A tree can be seen as a piecewise constant approximation.

In [ ]:
tree.DecisionTreeClassifier?

In [ ]:
_seed = generate_seed()

_classifier_hyperparameters = {
    "random_state": np.random.RandomState(_seed)
}

estimator = tree.DecisionTreeClassifier(**_classifier_hyperparameters)

# here we fit using the raw training features
_ = estimator.fit(train_features, train_target)

In [ ]:
# make predictions
_predictions = estimator.predict(train_features)

# report the accuracy on the training data
_report = metrics.classification_report(
    train_target,
    _predictions,
)
print(_report)

Wait, what!? No error at all? Could this model really be absolutely perfect? Unfortunately it is much more likely that the model has badly overfit the training data. How can you be sure? As we saw earlier, you don’t want to touch the testing dataset until you are ready to launch a model you are confident about, so you need to use part of the training set for training and part of it for model validation.

The following code use Scikit-Learn [`model_selection.cross_val_score`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score) to randomly split the training set into 5 distinct subsets called folds, then it trains and evaluates our model 5 times, picking a different fold for evaluation every time and training on the other 4 folds. The result is an array containing the 5 evaluation scores.

In [ ]:
estimator_scores = model_selection.cross_val_score(
    estimator,
    X=train_features,
    y=train_target,
    cv=CV_FOLDS,
    scoring="accuracy",
    n_jobs=-1,
    verbose=1
)

In [ ]:
estimator_scores.mean()

### Understanding Feature Importance

One of the nice features of decision trees is that they provide a way to measure the importance of each of feature. Understanding feature importance is a topic all unto itself. If you are interested in pulling this thread, then I recommend that you start with [SHapley Additive Explanations (SHAP)](https://shap.readthedocs.io/en/latest/index.html) and then take a look through [*Interpretable Machine Learning*](https://christophm.github.io/interpretable-ml-book/).

In [ ]:
estimator.feature_importances_.min(), estimator.feature_importances_.mean(), estimator.feature_importances_.max()

In [ ]:
is_positive = estimator.feature_importances_ > 0
is_positive.sum() / estimator.feature_importances_.size

Because our features are pixels we can reshape and plot the features to gain some insight into what might be driving feature importance.

In [ ]:
_, ax = plt.subplots(1, 1, figsize=(15, 15))
_average_pixel_importances = (
    estimator.feature_importances_
             .reshape((3, 32, 32))
             .mean(axis=0)
)
plt.imshow(_average_pixel_importances)
plt.title("Average Pixel Importance", fontsize=20)

# create a colorbar
colorbar = plt.colorbar(ticks=[_average_pixel_importances.min(), _average_pixel_importances.max()])
_ = (colorbar.ax
             .set_yticklabels(["Not important", "Very important"], fontsize=15))

The majority of features seem to be unimportant which suggests that perhaps we should look at ways to extract the most important features prior to fitting our decision tree model. The code below creates a preprocessing pipeline that uses PCA to reduce the dimension of the feature space and extract more meaningful input features and retrain a decision tree classifier. Does this improve the results?

In [ ]:
# use PCA to reduce dimensionality and standardize features
_seed = generate_seed()
_pca_hyperparameters = {
    "n_components": 250,
    "random_state": np.random.RandomState(_seed),
    "svd_solver": "randomized",
    "whiten": False,
}
_preparation_pipeline = pipeline.make_pipeline(
    decomposition.PCA(**_pca_hyperparameters),
    verbose=True,
)
prepared_train_features = _preparation_pipeline.fit_transform(train_features)

# fit decision tree classifier
estimator = tree.DecisionTreeClassifier(random_state=_random_state,)
_ = estimator.fit(prepared_train_features, train_target)

# generate a classification report
_predictions = estimator.predict(prepared_train_features)
_report = metrics.classification_report(
    train_target,
    _predictions,
)
print(_report)

### Exercise: Regularizing Decision Trees

Our decision tree classifier appears to be overfitting. We need to regularize it. Decreasing `max_*` parameters and increasing `min_*` parameters will increase the amount of regularization applied to the model and will help reduce overfitting.

In [ ]:
tree.DecisionTreeClassifier?

In [ ]:
# fit decision tree classifier
_seed = generate_seed()
_classifier_hyperparameters = {
    "criterion": "entropy",
    "max_depth": 32,
    "max_features": "sqrt",
    "min_samples_leaf": 1e-3,
    "min_samples_split": 2e-3,
    "random_state": np.random.RandomState(_seed)
}
estimator = tree.DecisionTreeClassifier(**_classifier_hyperparameters)
_ = estimator.fit(prepared_train_features, train_target)

# make predictions
_predictions = estimator.predict(prepared_train_features)

# create a classification report
_report = metrics.classification_report(
    train_target,
    _predictions,
)
print(_report)

In [ ]:
estimator_scores = model_selection.cross_val_score(
    estimator,
    X=prepared_train_features,
    y=train_target,
    cv=CV_FOLDS,
    scoring="accuracy",
    n_jobs=-1
)

In [ ]:
estimator_scores.mean()

### Exercise

Compute the normalized confusion matrix for your decision tree classifier and plot it. Do you notice any patterns? What are the three classes for which your decision tree classifier peforms the worst?

In [ ]:
# insert code here!

### Ensemble Methods

Building a model on top of many other models is called [ensemble](https://scikit-learn.org/stable/modules/ensemble.html) learning and it is often a great approach to improve the predictions of your machine learning pipeline.

#### Random Forests

Let’s try the [`ensemble.RandomForestRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html). Random forests work by training many decision trees on random subsets of the features, then averaging the predictions made by each of the decision trees to arrive at an overall prediction.

In [ ]:
ensemble.RandomForestClassifier?

In [ ]:
_seed = generate_seed()
_classifier_hyperparameters = {
    "n_jobs": -1,
    "random_state": np.random.RandomState(_seed)
}

estimator = ensemble.RandomForestClassifier(**_classifier_hyperparameters)
_ = estimator.fit(train_features, train_target)

In [ ]:
# make predictions
_predictions = estimator.predict(train_features)

# generate a classification report
_report = metrics.classification_report(
    train_target,
    _predictions,
)
print(_report)

##### Visualizing Feature Importance

In [ ]:
_, ax = plt.subplots(1, 1, figsize=(15, 15))
_average_pixel_importances = (
    estimator.feature_importances_
             .reshape((3, 32, 32))
             .mean(axis=0)
)
plt.imshow(_average_pixel_importances)
plt.title("Average Pixel Importance", fontsize=20)

# create a colorbar
colorbar = plt.colorbar(ticks=[_average_pixel_importances.min(), _average_pixel_importances.max()])
_ = (colorbar.ax
             .set_yticklabels(["Not important", "Very important"], fontsize=15))

##### Measuring Validation Error

Again you can use $k$-fold CV to estimate the validation error for your random forest classifier.

In [ ]:
_seed = generate_seed()
_classifier_hyperparameters = {
    "n_jobs": -1,
    "random_state": np.random.RandomState(_seed)
}

estimator = ensemble.RandomForestClassifier(**_classifier_hyperparameters)

estimator_scores = model_selection.cross_val_score(
    estimator,
    X=prepared_train_features, # use PCA preprocessed features to speed things up!
    y=train_target,
    cv=CV_FOLDS,
    scoring="accuracy",
    n_jobs=-1,
    verbose=1,
)

In [ ]:
estimator_scores.mean()

Alternatively, with random forests, you can avoid CV altogether by only using a random subset of training samples to fit each tree. The unused training samples can then be used to estimate the validation error for each tree.

In [ ]:
_seed = generate_seed()
_classifier_hyperparameters = {
    "bootstrap": True,
    "max_samples": 0.9,
    "n_jobs": -1,
    "oob_score": True,
    "random_state": np.random.RandomState(_seed)
}

estimator = ensemble.RandomForestClassifier(**_classifier_hyperparameters)
_ = estimator.fit(prepared_train_features, train_target)

In [ ]:
estimator.oob_score_

### Exercise: Regularizing Random Forests

Our random forest classifier is still overfitting (but perhaps not quite as badly as the decision tree classifier). Can tune the behavior of each tree using the same tuning parameters as above. Also can control the number of estimators used in constructing the ensemble: more estimators means a more flexible model. The Scikit Learn documentation has a good discussion on [parameter tunning strategies](https://scikit-learn.org/stable/modules/ensemble.html#random-forest-parameters) for random forest classifiers and regressors. Manually tune the hyperparameters of the random forest classifier to train and find a good set of hyperparameters.

In [ ]:
_seed = generate_seed()
_classifier_hyperparameters = {
    "criterion": "entropy",
    "max_depth": 16,
    "max_features": "sqrt",
    "max_samples": 0.9,
    "min_samples_leaf": 1e-3,
    "min_samples_split": 2e-3,
    "n_estimators": 250,
    "bootstrap": True,
    "max_samples": 0.9,
    "n_jobs": -1,
    "oob_score": True,
    "random_state": np.random.RandomState(_seed)
}
estimator = ensemble.RandomForestClassifier(**_classifier_hyperparameters)
_ = estimator.fit(prepared_train_features, train_target)

In [ ]:
# create a classification report
_predictions = estimator.predict(prepared_train_features)
_report = metrics.classification_report(
    train_target,
    _predictions,
)
print(_report)

In [ ]:
estimator.oob_score_

### Exercise: Exploring Gradient Boosted Trees

Read the docs for to understand default behavior of the [`ensemble.HistGradientBoostingClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html). Fit a gradient boosted classifier and manually tune the hyperparameters and see if you can outperform your random forest classifier.

In [ ]:
_seed = generate_seed()
_classifier_hyperparameters = {
    "max_iter": 100,
    "random_state": np.random.RandomState(_seed),
    "scoring": "accuracy",
    "tol": 1e-3
}

estimator = ensemble.HistGradientBoostingClassifier(**_classifier_hyperparameters)
_ = estimator.fit(prepared_train_features, train_target)

In [ ]:
# make predictions
_predictions = estimator.predict(prepared_train_features)

# generate a classification report
_report = metrics.classification_report(
    train_target,
    _predictions,
)
print(_report)

In [ ]:
estimator_scores = model_selection.cross_val_score(
    estimator,
    X=prepared_train_features,
    y=train_target,
    cv=CV_FOLDS,
    scoring="accuracy",
    n_jobs=-1,
    verbose=1,
)

In [ ]:
estimator_scores.mean()

# Fine-tune your models

Most common approach to tuning a model is to manually fiddle with the hyperparameters until you find a great combination of hyperparameter values. Needless to day, this approach to model tuning is very tedious and not at all scientific. We can do much better!

## Grid Search

Simplest approach is to use Scikit-Learn’s [`model_selection.GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). All you need to do is tell it which hyperparameters you want it to experiment with and what values to try out. The `model_selection.GridSearchCV` class will then use cross-validation to evaluate all the possible combinations of hyperparameter values and return the best scoring set of hyperparameters according to your specified metric.

In [ ]:
# hyperparameters that you don't want to tune
_seed = generate_seed()
_pca_default_hyperparameters = {
    "random_state": np.random.RandomState(_seed),
    "svd_solver": "randomized",
    "whiten": False,
}

_seed = generate_seed()
_classifier_default_hyperparameters = {
    "bootstrap": False,
    "oob_score": False,
    "max_samples": None,
    "n_jobs": -1,
    "random_state": np.random.RandomState(_seed),
}

_pipeline = pipeline.make_pipeline(
    decomposition.PCA(
        **_pca_default_hyperparameters
    ),
    ensemble.RandomForestClassifier(
        **_classifier_default_hyperparameters
    ),
    verbose=True,
    memory="../results/models", # enables on-disk caching!
)

_parameter_grid = [
    { # small number of deep trees
        "pca__n_components": [25],
        "randomforestclassifier__max_depth": [None],
        "randomforestclassifier__max_features": [None],
        "randomforestclassifier__n_estimators": [2, 4, 8, 16],
        "randomforestclassifier__min_samples_leaf": [1],
        "randomforestclassifier__min_samples_split": [2]
    }, # 1 * 1 * 1 * 4 * 1 * 1 = 4 parameter combinations to try
    { # large number of short trees
        "pca__n_components": [25],
        "randomforestclassifier__max_depth": [None],
        "randomforestclassifier__max_features": ["log2"],
        "randomforestclassifier__n_estimators": [100, 200, 400, 800],
        "randomforestclassifier__min_samples_leaf": [10, 100, 1000],
    } # 1 * 1 * 1 * 4 * 3 = 12 parameter combinations to try
] # 4 + 12 = 16 total parameter combinations to try

estimator = model_selection.GridSearchCV(
    _pipeline,
    _parameter_grid,
    cv=CV_FOLDS, # 3 * 16 = 48 total fits!
    scoring="accuracy",
    return_train_score=True,
    n_jobs=-1,
    pre_dispatch=2, # important to set this properly to avoid OOM errors
    verbose=1,
)

In [ ]:
_ = estimator.fit(train_features, train_target)

In [ ]:
estimator.best_score_

In [ ]:
estimator.best_params_

In [ ]:
estimator.best_estimator_

You should save every model you experiment with so that you can come back easily to any model you want. Make sure you save both the hyperparameters and the trained parameters as well as the cross-validation scores and perhaps the actual predictions as well. This will allow you to more easily compare scores across model types and compare the types of errors they make.

In [ ]:
RESULTS_DIR = pathlib.Path("../results")
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

timestamp = time.strftime("%Y%m%d-%H%M%S")
_ = joblib.dump(estimator, RESULTS_DIR / f"grid-search-cv-classifier-{timestamp}.pkl")

For reference here is how you would reload the trained model from the file.

In [ ]:
reloaded_estimator = joblib.load(RESULTS_DIR / f"grid-search-cv-classifier-{timestamp}.pkl")

In [ ]:
reloaded_estimator.best_params_

## Randomized Search

The grid search approach is fine when you are exploring relatively few combinations but when the hyperparameter search space is large it is often preferable to use [`model_selection.RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV) instead. Instead of trying out all possible combinations, `model_selection.RandomizedSearchCV` evaluates a given number of random combinations by selecting a random value for each hyperparameter at every iteration. This approach has two main benefits.

* More efficient exploration of the hyperparameter space.
* More control over the computing budget you want to allocate to hyperparameter search.

Cost is that it requires some fairly detailed knowledge of probability distributions to implement as you need to choose probability distributions for each hyperparameter that you wish to tune.

### Geometric Discrete Distribution

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
x = np.arange(1, 1000, 1)
y = (stats.geom
          .pmf(x, p=0.005))
ax.plot(x, y, 'o', alpha=0.05)
ax.set_xscale("log")
ax.set_xlabel("x")
ax.set_ylabel("Probability Mass")
plt.show()

In [ ]:
(stats.geom(p=0.005)
      .rvs(100))

### Beta Continuous Distribution

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
x = np.linspace(0, 1, 1000)
y = (stats.beta
          .pdf(x, a=25, b=2))
ax.plot(x, y)
ax.set_xlabel("x")
ax.set_ylabel("Probability Mass")
plt.show()

In [ ]:
(stats.beta(a=100, b=10)
      .rvs(100))

### Exercise: Playing with probability distributions

Play around with the parameters of the distributions above and see if you can get a feel for how the shape of the pmf/pdf changes with different parameter values.

In [ ]:
_seed = generate_seed()
_pca_default_hyperparameters = {
    "random_state": np.random.RandomState(_seed),
    "whiten": False,
}

_seed = generate_seed()
_classifier_default_hyperparameters = {
    "bootstrap": False,
    "oob_score": False,
    "max_samples": None,
    "n_jobs": -1,
    "random_state": np.random.RandomState(_seed),
}

_pipeline = pipeline.make_pipeline(
    decomposition.PCA(
        **_pca_default_hyperparameters
    ),
    ensemble.RandomForestClassifier(
        **_classifier_default_hyperparameters
    ),
    verbose=True,
    memory="../results/models", # enables on-disk caching!
)


_parameter_distributions = [
    {
        "pca__n_components": stats.beta(a=100, b=10),
        "randomforestclassifier__n_estimators": stats.geom(p=0.005),
        "randomforestclassifier__min_samples_split": stats.beta(a=2, b=100),
    }
]

_seed = generate_seed()
estimator = model_selection.RandomizedSearchCV(
    _pipeline,
    _parameter_distributions,
    cv=CV_FOLDS,
    n_iter=10, # 3 * 10 = 30 total fits!
    n_jobs=-1,
    pre_dispatch=2, # important to set this properly to avoid OOM errors
    random_state=np.random.RandomState(_seed),
    scoring="accuracy",
    verbose=1
)

In [ ]:
_ = estimator.fit(train_features, train_target)

In [ ]:
estimator.best_score_

In [ ]:
estimator.best_params_

In [ ]:
timestamp = time.strftime("%Y%m%d-%H%M%S")
_ = joblib.dump(estimator, RESULTS_DIR / f"randomized-search-cv-classifier-{timestamp}.pkl")

### Exercise:

Fine-tune one of your models using Grid Search; fine-tune another model using Randomized Search.

In [ ]:
# insert your code here!

# Evaluate your models on the test dataset

After tweaking your models for a while, you eventually have a system that performs sufficiently well. Now is the time to evaluate the final model on the test set.

In [ ]:
# make predictions
predictions = estimator.predict(test_features)

# generate a classification report
_report = metrics.classification_report(
    test_target,
    predictions,
)
print(_report)

If you did a lot of hyperparameter tuning, the performance will usually be slightly worse than what you measured using cross-validation (because your system ends up fine-tuned to perform well on the validation data and will likely not perform as well on unknown datasets). It is not the case in this example, but when this happens you must resist the temptation to tweak the hyperparameters to make the numbers look good on the test set; the improvements would be unlikely to generalize to new data.